In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time
import os
import copy
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
import cv2
import numpy as np
import imageio as io
from PIL import Image
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
class_names = ['Abstract_Expressionism','Baroque','Cubism', 'Fauvism', 'Impressionism','Minimalism','Naive_Art_Primitivism','Pointillism','Rococo','Ukiyo_e']
device = 'cuda'
!pip install torchsampler
class ArtStyleDataset(Dataset):
  def __init__(self, root_dir, transform = None, transform_tensor = None):
     self.imgs = ImageFolder(root_dir)
     self.root_dir = root_dir
     self.transform = transform
     self.transform_tensor = transform_tensor


  def __len__(self):
        return len(self.imgs)

  def __getitem__(self, idx):
        sample, label = self.imgs[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label

transform = transforms.Compose([
    transforms.Resize((229, 229)),
    transforms.ToTensor(),
])
#load data
data = ArtStyleDataset('wiki-art-selected-resized-downsampled', transform)

from torch.utils.data import Dataset, DataLoader, random_split
train_size = int(0.8 * len(data))
test_size = int(0.2 * len(data))
#val_size = len(data) - train_size - test_size
train_dataset, test_dataset = random_split(data, [train_size, test_size])

train_loader = DataLoader(train_dataset, shuffle=True,num_workers=4)
test_loader = DataLoader(test_dataset, shuffle=False, num_workers=4)
#val_loader = DataLoader(val_dataset, shuffle=False, num_workers=4)

from torch.utils.data import WeightedRandomSampler

def extract_labels():
  features = []
  labels_ = []
  map_img_feature = {}
  for inputs, labels in train_loader:
      labels_.append(labels.numpy())
  return labels_
y = extract_labels()
y = np.concatenate(y)
unique, counts = np.unique(y, return_counts=True)


#Up sampling
class_weights = [1.0/c for c in counts]
sample_weights = [class_weights[i] for i in y]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

train_loader = DataLoader(train_dataset,sampler=sampler, batch_size = 32)
dataset_sizes= {"train" : len(train_loader), "test" : len(test_loader)}



Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [5]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torchmetrics
from torchvision import models

# Initialize metrics storage lists
f1_mobilenet = []
accuracy_mobilenet = []
recall_mobilenet = []
precision_mobilenet = []

class MobileNetV20(pl.LightningModule):
    def __init__(self, num_classes):
        super(MobileNetV20, self).__init__()
        # Load pretrained MobileNetV2
        self.model = models.mobilenet_v2(pretrained=True)
        self.validation_step_outputs = []
        self.train_step_outputs = []

        # Freeze pre-trained layers
        for param in self.model.features.parameters():
            param.requires_grad = False

        self.model.classifier = nn.Sequential(
            nn.Linear(1280, 256),  # Normalize to 256 features
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),  # Final layer
            nn.LogSoftmax(dim=1)          # LogSoftmax for classification
        )


        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes, average='weighted')
        self.val_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes, average='weighted')
        self.f1_val = torchmetrics.F1Score(task="multiclass", num_classes=num_classes, average='weighted')
        self.f1_train = torchmetrics.F1Score(task="multiclass", num_classes=num_classes, average='weighted')
        self.precision_val = torchmetrics.Precision(task="multiclass", num_classes=num_classes, average='weighted')
        self.recall_val = torchmetrics.Recall(task="multiclass", num_classes=num_classes, average='weighted')

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = nn.CrossEntropyLoss()(logits, y)
        preds = torch.argmax(logits, dim=1)

        self.accuracy.update(preds, y)
        self.f1_train.update(preds, y)

        self.train_step_outputs.append(loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        val_loss = nn.CrossEntropyLoss()(logits, y)

        preds = torch.argmax(logits, dim=1)

        self.val_accuracy(preds, y)
        self.f1_val(preds, y)
        self.precision_val(preds, y)
        self.recall_val(preds, y)

        self.val_accuracy.update(preds, y)
        self.f1_val.update(preds, y)
        self.precision_val.update(preds, y)
        self.recall_val.update(preds, y)

        self.validation_step_outputs.append(val_loss)

    def on_train_epoch_end(self):
        train_acc = self.accuracy.compute()
        epoch_f1_train = self.f1_train.compute()
        print(f"Epoch {self.current_epoch}: Train Accuracy: {train_acc:.4f} F1: {epoch_f1_train:.4f}")
        self.accuracy.reset()
        self.f1_train.reset()

    def on_validation_epoch_end(self):
        epoch_val_acc = self.val_accuracy.compute()
        avg_val_loss = torch.stack(self.validation_step_outputs).mean()
        epoch_f1_val = self.f1_val.compute()
        epoch_precision_val = self.precision_val.compute()
        epoch_recall_val = self.recall_val.compute()

        print(f"Epoch {self.current_epoch}: Validation Accuracy: {epoch_val_acc:.4f} F1: {epoch_f1_val:.4f}  Recall: {epoch_recall_val:.4f}  Precision: {epoch_precision_val:.4f}")
        self.log("validation_epoch_accuracy", epoch_val_acc)
        self.log("validation_epoch_loss", avg_val_loss)
        self.log("f1_val", epoch_f1_val)

        f1_mobilenet.append(epoch_f1_val.cpu().numpy())
        accuracy_mobilenet.append(epoch_val_acc.cpu().numpy())
        recall_mobilenet.append(epoch_recall_val.cpu().numpy())
        precision_mobilenet.append(epoch_precision_val.cpu().numpy())

        self.val_accuracy.reset()
        self.f1_val.reset()
        self.precision_val.reset()
        self.recall_val.reset()

        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        # Optimize only the new classifier layers
        optimizer = torch.optim.Adam(self.model.classifier.parameters(), lr= 0.0001)
        return optimizer


In [7]:
checkpoint_callbackS = ModelCheckpoint(

     monitor='f1_val',
       mode='max',
     dirpath='my/path2/',
     filename='mobnetv2S-{epoch:02d}-{f1_val:.2f}'
)

trainerS = pl.trainer.Trainer(
    max_epochs=25,
   accelerator="cuda",
    callbacks =checkpoint_callbackS,
)

modelResS = MobileNetV20(num_classes=10)

trainerS.fit(modelResS, train_loader, test_loader, ckpt_path=None)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/venv/main/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 29.1MB/s]
You are using a CUDA device ('NVIDIA GeForce RTX 3060') tha

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch 0: Validation Accuracy: 0.5000 F1: 0.3333  Recall: 0.5000  Precision: 0.2500


/venv/main/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 23: Validation Accuracy: 0.7344 F1: 0.7315  Recall: 0.7344  Precision: 0.7360
Epoch 23: Train Accuracy: 0.7690 F1: 0.7679


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: Validation Accuracy: 0.7309 F1: 0.7276  Recall: 0.7309  Precision: 0.7293
Epoch 24: Train Accuracy: 0.7728 F1: 0.7719


In [ ]:
mobnetv2S-epoch=23-f1_val=0.73

In [8]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torchmetrics
from torchvision import models

# Initialize metrics storage lists
f1_mobilenet = []
accuracy_mobilenet = []
recall_mobilenet = []
precision_mobilenet = []

class MobileNetV2(pl.LightningModule):
    def __init__(self, num_classes):
        super(MobileNetV2, self).__init__()
        # Load pretrained MobileNetV2
        self.model = models.mobilenet_v3_large(pretrained=True)
        self.validation_step_outputs = []
        self.train_step_outputs = []

        # Freeze pre-trained layers
        for param in self.model.features.parameters():
            param.requires_grad = False

        self.model.classifier = nn.Sequential(
            nn.Linear(960, 256),  # Normalize to 256 features
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),  # Final layer
            nn.LogSoftmax(dim=1)          # LogSoftmax for classification
        )


        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes, average='weighted')
        self.val_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes, average='weighted')
        self.f1_val = torchmetrics.F1Score(task="multiclass", num_classes=num_classes, average='weighted')
        self.f1_train = torchmetrics.F1Score(task="multiclass", num_classes=num_classes, average='weighted')
        self.precision_val = torchmetrics.Precision(task="multiclass", num_classes=num_classes, average='weighted')
        self.recall_val = torchmetrics.Recall(task="multiclass", num_classes=num_classes, average='weighted')

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = nn.CrossEntropyLoss()(logits, y)
        preds = torch.argmax(logits, dim=1)

        self.accuracy.update(preds, y)
        self.f1_train.update(preds, y)

        self.train_step_outputs.append(loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        val_loss = nn.CrossEntropyLoss()(logits, y)

        preds = torch.argmax(logits, dim=1)

        self.val_accuracy(preds, y)
        self.f1_val(preds, y)
        self.precision_val(preds, y)
        self.recall_val(preds, y)

        self.val_accuracy.update(preds, y)
        self.f1_val.update(preds, y)
        self.precision_val.update(preds, y)
        self.recall_val.update(preds, y)

        self.validation_step_outputs.append(val_loss)

    def on_train_epoch_end(self):
        train_acc = self.accuracy.compute()
        epoch_f1_train = self.f1_train.compute()
        print(f"Epoch {self.current_epoch}: Train Accuracy: {train_acc:.4f} F1: {epoch_f1_train:.4f}")
        self.accuracy.reset()
        self.f1_train.reset()

    def on_validation_epoch_end(self):
        epoch_val_acc = self.val_accuracy.compute()
        avg_val_loss = torch.stack(self.validation_step_outputs).mean()
        epoch_f1_val = self.f1_val.compute()
        epoch_precision_val = self.precision_val.compute()
        epoch_recall_val = self.recall_val.compute()

        print(f"Epoch {self.current_epoch}: Validation Accuracy: {epoch_val_acc:.4f} F1: {epoch_f1_val:.4f}  Recall: {epoch_recall_val:.4f}  Precision: {epoch_precision_val:.4f}")
        self.log("validation_epoch_accuracy", epoch_val_acc)
        self.log("validation_epoch_loss", avg_val_loss)
        self.log("f1_val", epoch_f1_val)

        f1_mobilenet.append(epoch_f1_val.cpu().numpy())
        accuracy_mobilenet.append(epoch_val_acc.cpu().numpy())
        recall_mobilenet.append(epoch_recall_val.cpu().numpy())
        precision_mobilenet.append(epoch_precision_val.cpu().numpy())

        self.val_accuracy.reset()
        self.f1_val.reset()
        self.precision_val.reset()
        self.recall_val.reset()

        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        # Optimize only the new classifier layers
        optimizer = torch.optim.Adam(self.model.classifier.parameters(), lr= 0.0001)
        return optimizer


In [10]:
checkpoint_callbackS = ModelCheckpoint(

     monitor='f1_val',
       mode='max',
     dirpath='my/path2/',
     filename='mobnetv3_new25.03S-{epoch:02d}-{f1_val:.2f}'
)

trainerS = pl.trainer.Trainer(
    max_epochs=25,
   accelerator="cuda",
    callbacks =checkpoint_callbackS,
)

modelResS = MobileNetV2(num_classes=10)

trainerS.fit(modelResS, train_loader, test_loader, ckpt_path=None)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/venv/main/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 39.2MB/s]
/venv/main/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /my/path2 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                | Params | Mode 
------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch 0: Validation Accuracy: 0.0000 F1: 0.0000  Recall: 0.0000  Precision: 0.0000


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 23: Validation Accuracy: 0.7490 F1: 0.7471  Recall: 0.7490  Precision: 0.7509
Epoch 23: Train Accuracy: 0.7862 F1: 0.7853


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: Validation Accuracy: 0.7541 F1: 0.7509  Recall: 0.7541  Precision: 0.7547
Epoch 24: Train Accuracy: 0.7933 F1: 0.7922


In [8]:
from google.colab import files
files.download('/content/my/path/mobnetS-epoch=14-f1_val=0.74.ckpt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install --upgrade pytorch-lightning


In [1]:
def evaluate_model(predicted, true):
  from sklearn.metrics import classification_report
  import pandas as pd
  from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
  import matplotlib.pyplot as plt
  report_dict = classification_report(predicted, true, target_names=class_names, output_dict=True)
  metrics_df2 = pd.DataFrame(report_dict).transpose()
  print(metrics_df2)

  disp = ConfusionMatrixDisplay(confusion_matrix(true, predicted, normalize = 'true' )*100, display_labels=class_names)

  fig, ax = plt.subplots(figsize=(8, 6))
  disp.plot(cmap="cividis", ax=ax) 
  for _, spine in ax.spines.items():
      spine.set_visible(False)  
  plt.xticks(rotation=90, ha="right")
  plt.yticks(rotation=0)
  plt.grid(False) 
  plt.title("Confusion Matrix")
    #plt.colorbar(ax.collections[0], ax=ax)  
  disp = ConfusionMatrixDisplay(confusion_matrix(true, predicted), display_labels=class_names)

  fig, ax = plt.subplots(figsize=(8, 6))
  disp.plot(cmap="cividis", ax=ax) 
  for _, spine in ax.spines.items():
        spine.set_visible(False)  

  plt.xticks(rotation=90, ha="right")
  plt.yticks(rotation=0)
  plt.grid(False) 
  plt.title("Not normalized Confusion Matrix")
  plt.show()


def validate_model(model, data_loader):
    pred = []
    true = []
    model.eval()

    start_time = time.time()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs.float())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            true.extend(labels.cpu().numpy())
            pred.extend(predicted.cpu().numpy())

        accuracy = 100 * correct / total  # Calculate accuracy based on predictions

        print('Finished Testing')
        print(f'Testing accuracy: {accuracy:.1f} %')

    evaluate_model(pred, true)